In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import cv2
import pytesseract
import requests
import os
from word2number import w2n
import re

# Khởi tạo trình duyệt
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Mở trang CAPTCHA
driver.get("URL_CỦA_TRANG_CAPTCHA")  # Cập nhật URL thực tế

# Đợi CAPTCHA hiển thị (tùy trang web có thể cần điều chỉnh thời gian)
time.sleep(3)

# Tìm ảnh CAPTCHA
captcha_img = driver.find_element(By.XPATH, "//img[contains(@src, 'captcha')]")  # Điều chỉnh XPATH nếu cần
captcha_url = captcha_img.get_attribute("src")

# Tải ảnh về máy
captcha_path = "captcha.jpg"
response = requests.get(captcha_url)
with open(captcha_path, "wb") as file:
    file.write(response.content)

# Đọc ảnh CAPTCHA
image = cv2.imread(captcha_path)

# Tiền xử lý ảnh để cải thiện OCR
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

# Nhận diện văn bản từ ảnh
text = pytesseract.image_to_string(thresh)
print("Văn bản trích xuất:", text)

# Tìm số trong văn bản (bao gồm cả dạng chữ)
words = text.lower().split()
number_list = []

for word in words:
    if word.isdigit():
        number_list.append(int(word))
    else:
        try:
            number_list.append(w2n.word_to_num(word))
        except ValueError:
            pass  # Bỏ qua nếu không chuyển đổi được

# Lấy số nhỏ hơn trong danh sách
if len(number_list) >= 2:
    lower_number = min(number_list)
    print("Số cần chọn:", lower_number)
else:
    print("Không tìm thấy đủ số hợp lệ!")
    driver.quit()
    exit()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="wt_captcha_slider_text"]"}
  (Session info: chrome=134.0.6998.118); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6A2094C25+3179557]
	(No symbol) [0x00007FF6A1CF88A0]
	(No symbol) [0x00007FF6A1B891CA]
	(No symbol) [0x00007FF6A1BDFA67]
	(No symbol) [0x00007FF6A1BDFC9C]
	(No symbol) [0x00007FF6A1C33627]
	(No symbol) [0x00007FF6A1C07C6F]
	(No symbol) [0x00007FF6A1C302F3]
	(No symbol) [0x00007FF6A1C07A03]
	(No symbol) [0x00007FF6A1BD06D0]
	(No symbol) [0x00007FF6A1BD1983]
	GetHandleVerifier [0x00007FF6A20F67CD+3579853]
	GetHandleVerifier [0x00007FF6A210D1D2+3672530]
	GetHandleVerifier [0x00007FF6A2102153+3627347]
	GetHandleVerifier [0x00007FF6A1E6092A+868650]
	(No symbol) [0x00007FF6A1D02FFF]
	(No symbol) [0x00007FF6A1CFF4A4]
	(No sym

In [ ]:
from selenium.webdriver.common.action_chains import ActionChains

# Tìm thanh trượt slider
slider = driver.find_element(By.CLASS_NAME, "slider-selector")  # Cập nhật class thực tế

# Di chuyển slider
action = ActionChains(driver)

# Điều chỉnh khoảng cách kéo (cần thử nghiệm)
move_distance = lower_number * 10
action.click_and_hold(slider).move_by_offset(move_distance, 0).release().perform()

# Nhấn nút Submit
submit_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Submit')]")
submit_button.click()

# Đợi một chút để kiểm tra kết quả
time.sleep(3)

# Đóng trình duyệt
driver.quit()


In [44]:
from selenium import webdriver
from selenium.common.exceptions import WebDriverException

In [ ]:
# Mở trình duyệt để nhập captcha
print("🔓 Mở trình duyệt để nhập captcha...")
driver = webdriver.Chrome()
driver.get("https://trade.ec.europa.eu/access-to-markets/en/search?product=0101&origin=VN&destination=AT")  # Thay bằng URL cần mở
print("⌨️ Vui lòng nhập captcha, sau đó đóng trình duyệt.")

# Đo thời gian bắt đầu
start_captcha_time = time.time()
wait_time = 600  # 10 phút (600 giây)

# Chờ người dùng tự đóng trình duyệt, nếu quá 10 phút thì tự động đóng
while True:
    try:
        if time.time() - start_captcha_time > wait_time:
            print("⏳ Quá 10 phút, tự động đóng trình duyệt.")
            driver.quit()
            break  # Thoát khỏi vòng lặp

        # Kiểm tra trình duyệt còn mở không
        driver.title  
        
        # Kiểm tra trang có bị treo không
        if driver.execute_script("return document.readyState") != "complete":
            print("⚠️ Trang web có thể bị treo, chờ thêm...")
        
    except WebDriverException:
        print("✅ Trình duyệt đã được đóng.")
        break  # Nếu trình duyệt bị đóng, thoát vòng lặp
    
    time.sleep(5)  # Tránh vòng lặp chạy quá nhanh, gây tải CPU

🔓 Mở trình duyệt để nhập captcha...
⌨️ Vui lòng nhập captcha, sau đó đóng trình duyệt.
